In [ ]:
import pandas as pd
import os
import numpy as np
from joblib import Parallel, delayed
from functools import reduce
import math as mt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
colname=['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'] 

In [ ]:
codephage=pd.read_csv('codephagefuncuniprot.csv')
codephage=codephage.sort_values('phage')
codephage['protindex']=codephage.groupby('phage').cumcount()+1

In [ ]:
dicto={}
for index,rows in codephage.iterrows():
    dicto[rows['code']]=rows['phage']
dicto2={}
for index,rows in codephage.iterrows():
    dicto2[rows['code']]=rows['protindex']

In [ ]:
phagenum=pd.read_csv('phagenumprotuniprotpandas.csv')
phagenum.columns=['scode','numprot','host']
phagenum2=phagenum[['scode','numprot']]

In [ ]:
for i in range(0,5):
    alpha=round(0.5+i*0.1,1)
    total={}
    for i in os.listdir():
        if i[0:5]=='bestm':
            recop={}
            a=pd.read_table(i, header=None)  #open results file
            a.columns=colname  ##assign columns
            #b=a.groupby('qseqid').head(1)  #top1
            b['scode']=b['sseqid'].apply(lambda x: dicto[x])
            b['numprot']=b['sseqid'].apply(lambda x: dicto2[x])
            c=b[['scode','numprot']].sort_values(['scode','numprot']) #works with hits only
            d=c.groupby(['scode','numprot']).size() #count reads by protein
            #counting packs
            for codigos in pd.unique(d.index.get_level_values(0)):
                recop[codigos]=0
                tmp=d[codigos].values
                while sum(tmp)!=0:
                    valores=np.heaviside(tmp,0)
                    if sum(valores)>=phagenum.loc[phagenum['scode']==codigos,'numprot'].values[0]*alpha:
                        recop[codigos]+=1
                    else:
                        break
                    tmp=tmp-1
                    tmp[tmp<0]=0
                    print(tmp)
            total[i[4:]]=recop
    #joint counts
    q=reduce(lambda x,y:pd.merge(x,y,how='outer',on='phage') ,[pd.DataFrame({'phage':list(y.keys()),x:list(y.values())}) for x,y in total.items()])
    qt=q.fillna(0).set_index('phage')
    qt2=qt.loc[:,qt.sum(axis=0)>0]
    up=qt2.loc[qt2.sum(axis=1)!=0]
    #up.to_csv("matrixphagesalpha"+str(alpha)+"newmethodmarineuniprot.csv") 
    xy=up.astype(bool).sum(axis=1)
    xy2=xy[xy!=0]
    xy3=pd.DataFrame({'prev':xy2}).reset_index()
    i=up.sum(axis=1).sort_values(ascending=False)
    i2=i[i!=0]
    i3=pd.DataFrame({'detec':i2}).reset_index()
    u=pd.merge(xy3,i3,how='outer',on='phage').sort_values('detec',ascending=False)
    #u.to_csv("phagesalpha"+str(alpha)+"newmethodmarineuniprot.csv",index=False)